In [ ]:
import pandas as pd
from pprint import pprint
from matplotlib import pyplot as plt
plt.style.use('seaborn-bright')

In [ ]:
%matplotlib inline

### Get the Iris data set

In [ ]:
# import the data as a pandas DataFrame object
data = pd.read_csv('/Users/hodges/Documents/Courses/SoftwareCarpentry/software-carpentry-sep2016/sci_python/iris_data.csv', index_col=0)

### Exploratory data analysis with `pandas`

Exploratory data analysis ([EDA](https://en.wikipedia.org/wiki/Exploratory_data_analysis)) aims at characterizing properties of a data set often using visualization techniques to see "what the data cen tell us", perhaps even to formulate new hypotheses and guide subsequent experiments. EDA is also key to assess which statistical models are appropriate in the sense that their assumptions are not violated and to explore whether data transformations are necessary or to explore issues like [confounding](https://en.wikipedia.org/wiki/Confounding). 

`pandas` provides plenty of methods of the `DataFrame` class for EDA, making it easy to get a quick overview of your data.

In [ ]:
dir(data)

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.head(3)

As we saw earlier, it's also very easy to produce simple plots directly from the dataframe. We will create some more detailed plots of the data, later on.

In [ ]:
data.plot('Sepal.Length','Sepal.Width',kind='scatter')

### Subsetting/filtering data

In [ ]:
# get True/False values for data matching specified criteria 
data['Species']=='setosa'

In [ ]:
# use these boolean vectors to filter the whole dataset
swidth_filter = data['Sepal.Width'] < 3.0
thin_sepals = data[swidth_filter]
print(thin_sepals.shape)
thin_sepals

In [ ]:
# get subsets of the data, for each species
seto = data[data['Species']=='setosa']
vers = data[data['Species']=='versicolor']
virg = data[data['Species']=='virginica']
print(seto.shape, vers.shape, virg.shape)

In [ ]:
# grouping data based on the values in a column can be achieved more easily with the group_by() method
data_by_species = data.groupby('Species')
for grp, grp_data in data_by_species:
    print(grp, grp_data.shape)
    print(grp_data.describe())

In [ ]:
# use the .loc() and .iloc() methods to specify coordinates of data that you want to subset
data.loc[5:10,'Species']

### Vector operations

In [ ]:
# you can operate on a whole series very quickly and easily
seto_pl = seto['Petal.Length']
print(seto_pl.head())
halved = seto_pl / 2
print(halved.head())

### More data summary methods

In [ ]:
for grp, grp_data in data.groupby('Species'):
    print(grp)
    print(grp_data['Petal.Width'].value_counts())
    

In [ ]:
box = data.boxplot()

You can do _so_ much more with `pandas` than we have the time to cover here. Check out the [documentation](http://pandas.pydata.org/pandas-docs/stable/index.html) to learn more about what's possible with this package, and see the more in-depth plotting examples below to get an idea of the approaches that you can take to visualise your data.

### More detailed exploration of differences between species

The following is inspired by http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html by Gaël Varoquaux.

In [ ]:
# Code source: Gaël Varoquaux
# Modified for documentation by Jaques Grobler
# License: BSD 3 clause

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

# separate the attribute columns from the class column (species)
X = data[['Sepal.Length',  'Sepal.Width',  'Petal.Length',  'Petal.Width']]
print(X.head())

# map string species names to numeric codes for classification
species_to_number = {'setosa': 0, 
                     'versicolor': 1, 
                     'virginica': 2}
y = np.array([ species_to_number[spec] for spec in data['Species'] ])
print(y)
classes = np.unique(y)
print(classes)

In [ ]:
# figure out plot coordinates
plt.figure(figsize=(8, 6))

# Plot the first two attributes of the iris data colored by class
plt.scatter(X['Sepal.Length'], X['Sepal.Width'], s=40, c=y, cmap=plt.cm.viridis)
plt.title('Two attributes of the Iris data', fontsize=14)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.show()

In [ ]:
# Explore separation of classes by individual attributes using boxplots
plt.figure(figsize=(8, 6))

plotdata = [X['Sepal.Length'][y == c] for c in classes]
bx = plt.boxplot(plotdata, notch=True, patch_artist=True)
for p in bx['boxes']:
    p.set_facecolor('lightgray')
    p.set_edgecolor('black')
for p in bx['whiskers']:
    p.set_color('black')
    p.set_linestyle('solid')    

plt.title('Attribute 0 (sepal length) broken down by class', fontsize=14)
plt.xlabel('Classes')
plt.ylabel('Sepal length')

# Overplot the means
plt.plot(classes+1, [np.average(d) for d in plotdata], '*',
             color='white', markeredgecolor='red', markersize=15)
plt.show()

### Investigate how different attributes are between classes more formally using statistical tests
In this section we demonstrate the application of several statistical tests (many more can be found in the scipy.stats package). Statistical tests are of course most useful, if we are not only interested in exploring the properties of our data, but also assess the statistical significane of (biologically) meaningful comparisons. 

In [ ]:
from scipy.stats import wilcoxon, kstest, f_oneway

# Apply Wilcoxon test to assess statistical significance of
# class-specific differences in sepal length (attribute 0)
z_statistic, wilcox_p_value = wilcoxon(X['Sepal.Length'], y)
print('Wilcoxon test P-value: %.3g'%wilcox_p_value)

# Let's also try a parameteric test, e.g. ANOVA 
f_statistic, anova_p_value = f_oneway(X['Sepal.Length'][y==0], X['Sepal.Length'][y==1], X['Sepal.Length'][y==2])
# f_statistic, anova_p_value = f_oneway(*[ X['Sepal.Length'][y==k] for k in set(y) ]) # alternative approach (don't miss out the * !)
print('ANOVA P-value: %.3g'%anova_p_value)

In [ ]:
# But wait a second: are the assumptions of the parametric ANOVA test met?
# Let's check with a Kolmogorov-Smirnov goodness-of-fit test whether
# the attribute values follow a Gaussian distribution (assumed by ANOVA).
fig, axarr = plt.subplots(1, 4, figsize=(20, 5))
for i in range(4):
    axarr[i].hist(X[X.columns[i]], 20, normed=1, facecolor='green', alpha=0.75)
    axarr[i].set_title('Histogram of %s' % X.columns[i], fontsize=14)
    d_statistic, ks_p_value = kstest(X[X.columns[i]], 'norm')
    axarr[i].text(min(X[X.columns[i]])+1, 0.2, 'K-S test P-value: %.3g'%ks_p_value,
         fontsize=14, ha='left')
plt.show()

In the sections above we investigated some properties of the Iris data to assess which statistical analysis approaches may be suitable. Histograms and the [Kolmogorov-Smirnov goodness-of-fit test](https://en.wikipedia.org/wiki/Kolmogorov%E2%80%93Smirnov_test) suggest that the attributes of the Iris data set do __not__ follow a Gaussian distribution, so we should be cautious with parametric statistical tests or models (such as [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test) or [ANOVA](https://en.wikipedia.org/wiki/Analysis_of_variance)) and rather apply non-parametric methods (such as the [Wilcoxon](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) or [Kruskal-Wallis tests](https://en.wikipedia.org/wiki/Kruskal%E2%80%93Wallis_one-way_analysis_of_variance)) that do not make distributional assumptions.

### Let's see whether we can cluster the Iris data set based on the first two attributes
For demonstration, we will use [k-means clustering](https://en.wikipedia.org/wiki/K-means_clustering) here, but there is many alternative approaches. A nice overview on how to quickly explore other algorithms available from scikit-learn can be found here: http://scikit-learn.org/stable/auto_examples/cluster/plot_cluster_comparison.html, this also provides a framework for visual evaluation of these algorithm.
Some example code was borrowed from http://stamfordresearch.com/k-means-clustering-in-python/ .

In [ ]:
from sklearn import cluster
from sklearn.metrics import accuracy_score

# Initialize, fit and predict with the K-Means clustering algorithm
np.random.seed(2016)

num_clust = 3
kmm = cluster.KMeans(n_clusters=num_clust)

# Let's only look at the first two attributes for a first demonstration
X_2d = X.iloc[:, :2]  
kmm.fit(X_2d)
y_pred = kmm.predict(X_2d)

# As the unsupervised K-means algorithm is unaware of the order of classes in Y,
# we need to (depending on random seed) re-arrange them to match class assignments
# between y and y_pred 
#print(y_pred)
#print(y)
y_pred = np.choose(kmm.labels_, [1, 2, 0]).astype(np.int64)
#print(y_pred)
#print(y)

In [ ]:
# Plot the clustering results

# To be able to re-use the plotting code, we encapsulate it in a function
def viz_clustering(ax, x_1, x_2, y, y_pred, centers=None):
    ax.scatter(x_1, x_2, s=40, c=y_pred, cmap=plt.cm.viridis)
    # Show cluster centers (means) if given
    if centers is not None:
        ax.scatter(centers[:, 0], centers[:, 1], linewidths=3, marker='x', s=300, color='black')

    # Highlight wrong cluster assignments
    ax.scatter(x_1[y != y_pred], x_2[y != y_pred], marker='o', s=70, 
               facecolors='none', edgecolors='red')

    # Evaluate prediction accuracy and annotate plot
    # Note that sklearn.metrics provides many alternative evaluation metrics
    acc = accuracy_score(y, y_pred)
    ax.text(max(x_1), max(x_2), 'Accuracy: '+str('%.3f'%acc), fontsize=14, ha='right')

    
plt.figure(figsize=(8, 6))
ax = plt.gca()

viz_clustering(ax, X['Sepal.Length'], X['Sepal.Width'], y, y_pred, centers=kmm.cluster_centers_[:,0:2])
plt.title('K-means clustering of the Iris data (first 2 attributes)', fontsize=14)
plt.xlabel('Sepal length')
plt.ylabel('Sepal width')

plt.show()

In [ ]:
# Let's repeat the exercise with all three available attributes.
np.random.seed(2016)
kmm.fit(X)
y_pred = kmm.predict(X)
y_pred = np.choose(kmm.labels_, [2, 0, 1]).astype(np.int64)

fig, axarr = plt.subplots(1, 3, figsize=(20, 6))
plt_cnt = 0
for i in classes[:-1]:
    for j in classes[classes > i]:
        #print(' i =',i,', j =',j)
        viz_clustering(axarr[plt_cnt], X[X.columns[i]], X[X.columns[j]], y, y_pred, 
                       centers=kmm.cluster_centers_[:,[i,j]])
        axarr[plt_cnt].set_title('K-means clustering of the whole Iris data', fontsize=14)
        axarr[plt_cnt].set_xlabel(X.columns[i])
        axarr[plt_cnt].set_ylabel(X.columns[j])
        plt_cnt += 1

plt.show()

### Run [PCA](https://en.wikipedia.org/wiki/Principal_component_analysis) as a means of dimensionality reduction
In very rough terms this will rotate (i.e. orthogonally transform) a high-dimensional data set so that the first principal components (PCs) will correspond to the largest sources of variance in the original data. Dimensionality reduction is typically achieved by projection onto the first PCs; in the example below, we only consider the first two PCs.
The code is again inspired by http://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html
by Gaël Varoquaux.

In [ ]:
from sklearn.decomposition import PCA

# We use PCA to project from the whole data set 
# onto the first two PCs to better visualize cluster separation
X_red = PCA(n_components=2).fit_transform(X) # returns a numpy.array object


In [ ]:
# Plot the first two principal components
plt.figure(figsize=(8, 6))

plt.scatter(X_red[:, 0], X_red[:, 1], s=40, c=y, cmap=plt.cm.viridis)
plt.title('PCA applied to the Iris data', fontsize=14)
plt.xlabel('PC1')
plt.ylabel('PC2')

plt.show()

### Let's (formally) test the separation between classes in the PCA space 
by application of k-means clustering to the first tow PCs and comparison of clustering accuracy to the approach above where k-means used the first two are all attributes of the original data.

In [ ]:
# Repeat the clustering on the result of the PCA
np.random.seed(2016)

# Verify that the PCA projection is indeed into a 2D space
print('Rank of PCA projection:',X_red.shape[1])

kmm.fit(X_red) # fit the clustering on the first two PCs
y_pred = kmm.predict(X_red)

# Again, match class assignments between y and y_pred 
#print(y_pred)
#print(y)
y_pred = np.choose(kmm.labels_, [2, 0, 1]).astype(np.int64)
#print(y_pred)
#print(y)

In [ ]:
# Visualize the clustering result (exactly as done above)
plt.figure(figsize=(8, 6))
ax = plt.gca()

viz_clustering(ax, X_red[:, 0], X_red[:, 1], y, y_pred, centers=kmm.cluster_centers_[:,0:2])
plt.title('K-means clustering applied to the first two PCs of the Iris data')
plt.xlabel('PC1')
plt.ylabel('PC2')

plt.show()


### Build a statistical classification model that recognizes the classes from the attributes
This section is based on a more exhaustive comparison of classification algorithms from the [scikit-learn documentation](http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html) by Gaël Varoquaux and Andreas Müller.

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
# As most training algorihtms have a stochastic component, setting the seed 
# of the random number generater is necessary for reproducibility
np.random.seed(2016)

# We will play with 4 rather popular classification algorihtms here
names = ['Nearest Neighbors', 'Linear SVM', 'Naive Bayes', 'Random Forest']
classifiers = [
    KNeighborsClassifier(5),
    SVC(kernel='linear', C=1),
    GaussianNB(),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
]
# note that these classifier initializations also set hyperparameters 
# which in real analysis scenarios need to be tuned using model selection

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

for name, clf in zip(names, classifiers):
    #ax = plt.subplot(len(datasets), len(classifiers) + 1, i)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print('%20s test accuracy: %.3f'%(name, acc))

In [ ]:
# For a visualization of classification results repeat the exercise
np.random.seed(2016)

# To be able to plot in 2 dimensions, restrict the data 
# to the attributes 1 and 2 and to classes 0 and 1
idx_attr = ['Sepal.Width', 'Petal.Length']
idx_exmp = y != 0
X_2d = X[idx_attr]
X_train, X_test, y_train, y_test = train_test_split(X_2d[idx_exmp], 
                                                    y[idx_exmp], test_size=.4)

# Construct a mesh for visualization of prediction scores
x_min, x_max = X_2d[idx_attr[0]].min() - 0.5, X_2d[idx_attr[0]].max() + 0.5
y_min, y_max = X_2d[idx_attr[1]].min() - 0.5, X_2d[idx_attr[1]].max() + 0.5
h = 0.02  # step size in the mesh
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# Prepare the plots (and color scheme)
fig = plt.figure(figsize=(27, 9))
plot_cnt = 1
cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

# Loop over classification algorithms and 
# visualize their decision boundary in a subplot for each
for name, clf in zip(names, classifiers):
    ax = plt.subplot(1, len(classifiers) + 1, plot_cnt)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    if hasattr(clf, "decision_function"):
        Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
    else:
        Z = clf.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

    # Visualize the predictions on the mesh as a color contour
    Z = Z.reshape(xx.shape)
    ax.contourf(xx, yy, Z, cmap=cm, alpha=.8)

    # Plot the training points on top
    ax.scatter(X_train[idx_attr[0]], X_train[idx_attr[1]], s=70, c=y_train, cmap=cm_bright, edgecolors='gray')
    # and testing points with white edges
    ax.scatter(X_test[idx_attr[0]], X_test[idx_attr[1]], s=70, c=y_test, cmap=cm_bright, edgecolors='white')

    ax.set_xlim(xx.min(), xx.max())
    ax.set_ylim(yy.min(), yy.max())
    ax.set_title(name)

    # Add a text label with the prediction accuracy on the test set
    acc = accuracy_score(y_test, y_pred)
    ax.text(xx.max() - .3, yy.min() + .3, ('Test acc. = %.3f' % acc).lstrip('0'),
            size=15, horizontalalignment='right')
    plot_cnt += 1

fig.subplots_adjust(left=.02, right=.98)
plt.show()


## Further Reading

- [`pandas` documentation & tutorials](http://pandas.pydata.org/pandas-docs/stable/index.html)
- [scikit-learn documentation & tutorials](http://scikit-learn.org/stable/)